This is a continuation from the `Tabular Keras` notebook.

In [26]:
import pandas as pd
import numpy as np
import keras
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Dense, BatchNormalization, Dropout, Input, LeakyReLU, PReLU
from keras.callbacks import LearningRateScheduler
from keras.optimizers import SGD
from keras.utils import to_categorical
from keras import backend as K

## Data preprocessing

In [2]:
def rebalance(frame, col='hab_lbl', factor=1):
    max_size = frame[col].value_counts().max()
    lst = [frame]
    for class_index, group in frame.groupby(col):
        lst.append(group.sample(int((max_size-len(group)) / factor), replace=True))
    frame_new = pd.concat(lst)
    
    return frame_new

In [3]:
df = pd.read_csv('dataset-rocky-no-STemp.csv')
df = df.drop('P. Habitable', axis=1)

In [4]:
train_df, test_df = train_test_split(df, train_size=0.7)

/home/rahul/.local/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [5]:
train_df = rebalance(train_df)

In [6]:
train_df.shape

(3501, 42)

In [7]:
y = train_df['hab_lbl']

In [8]:
y_test = test_df['hab_lbl']

In [9]:
train_df.drop('hab_lbl', axis=1, inplace=True)
test_df.drop('hab_lbl', axis=1, inplace=True)

/home/rahul/.local/lib/python3.5/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [10]:
y = to_categorical(np.array(y))
y_test = to_categorical(np.array(y_test))

## Model

In [11]:
x_train = np.array(train_df)
batch_size = 32

In [12]:
inp = Input(shape=(41,))

bn1 = BatchNormalization(name='first_bn')(inp)
relu = Dense(5, activation='relu', name='dense1')(bn1)
drop1 = Dropout(0.2, name='dropout1')(relu)

bn = BatchNormalization(name='bn1')(drop1)
relu = Dense(5, activation='relu', name='dense2')(bn)
drop2 = Dropout(0.2)(relu)

interm = keras.layers.Concatenate()([drop1, drop2])

bn = BatchNormalization(name='bn2')(interm)
relu = Dense(5, activation='relu', name='dense3')(bn)
drop = Dropout(0.2)(relu)

interm = keras.layers.Concatenate()([drop, drop2])

bn = BatchNormalization()(interm)
out = Dense(3, activation='softmax', name='dense4')(bn)

model = Model(inputs=inp, outputs=out)

In [13]:
func = K.function([model.layers[0].input], [model.layers[-2].output])

In [14]:
lrs = []

In [15]:
x_train.shape, y.shape

((3501, 41), (3501, 3))

In [16]:
def lr_schedule(epoch):
    """Learning Rate Schedule
    # Arguments
        epoch (int): The number of epochs
    # Returns
        lr (float32): learning rate
    """

    Kz = 0.
    for i in range((len(x_train) - 1) // batch_size + 1):
        start_i = i * batch_size
        end_i = start_i + batch_size
        xb = x_train[start_i:end_i]
    
        activ = np.linalg.norm(func([xb]))
        if activ > Kz:
            Kz = activ

    K_ = (2. * Kz) / (3. * batch_size)
    lr = 1 / K_
    lrs.append(lr)
    print('Epoch', epoch, 'LR =', lr)
    return lr
    #return K_ / np.sqrt(epoch + 1)

In [17]:
lr_scheduler = LearningRateScheduler(lr_schedule)

## Train with standard LR

In [18]:
model.compile(SGD(), loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
model.fit(x_train, y, validation_data=(np.array(test_df), y_test), epochs=10)

Train on 3501 samples, validate on 514 samples
Epoch 1/10
3501/3501 [==============================] - 3s 911us/step - loss: 0.9727 - acc: 0.5376 - val_loss: 0.2528 - val_acc: 0.9280
Epoch 2/10
3501/3501 [==============================] - 1s 216us/step - loss: 0.6933 - acc: 0.7241 - val_loss: 0.1950 - val_acc: 0.9397
Epoch 3/10
3501/3501 [==============================] - 1s 198us/step - loss: 0.5497 - acc: 0.7961 - val_loss: 0.1553 - val_acc: 0.9475
Epoch 4/10
3501/3501 [==============================] - 1s 204us/step - loss: 0.5031 - acc: 0.8129 - val_loss: 0.1364 - val_acc: 0.9494
Epoch 5/10
3501/3501 [==============================] - 1s 236us/step - loss: 0.4604 - acc: 0.8218 - val_loss: 0.1202 - val_acc: 0.9553
Epoch 6/10
3501/3501 [==============================] - 1s 232us/step - loss: 0.4483 - acc: 0.8175 - val_loss: 0.1044 - val_acc: 0.9591
Epoch 7/10
3501/3501 [==============================] - 1s 213us/step - loss: 0.4314 - acc: 0.8175 - val_loss: 0.1011 - val_acc: 0.9591
E

## Using LipschitzLR

In [20]:
model.compile(SGD(), loss='categorical_crossentropy', metrics=['accuracy'])

In [21]:
model.fit(x_train, y, validation_data=(np.array(test_df), y_test), epochs=10, callbacks=[lr_scheduler])

Train on 3501 samples, validate on 514 samples
Epoch 1/10
Epoch 0 LR = 1.6644086608850448
3501/3501 [==============================] - 4s 1ms/step - loss: 0.3586 - acc: 0.8535 - val_loss: 0.0870 - val_acc: 0.9844
Epoch 2/10
Epoch 1 LR = 0.8434567395433223
3501/3501 [==============================] - 1s 242us/step - loss: 0.2088 - acc: 0.9132 - val_loss: 0.0360 - val_acc: 0.9922
Epoch 3/10
Epoch 2 LR = 1.4378324789465708
3501/3501 [==============================] - 1s 262us/step - loss: 0.2072 - acc: 0.9077 - val_loss: 0.0377 - val_acc: 0.9922
Epoch 4/10
Epoch 3 LR = 1.2105283512667488
3501/3501 [==============================] - 1s 268us/step - loss: 0.2155 - acc: 0.9069 - val_loss: 0.0311 - val_acc: 0.9903
Epoch 5/10
Epoch 4 LR = 1.2339882941421874
3501/3501 [==============================] - 1s 246us/step - loss: 0.2027 - acc: 0.9057 - val_loss: 0.0447 - val_acc: 0.9864
Epoch 6/10
Epoch 5 LR = 0.7318742509912914
3501/3501 [==============================] - 1s 248us/step - loss: 0.169

# Leaky ReLU model

In [23]:
inp = Input(shape=(41,))

bn1 = BatchNormalization(name='first_bn')(inp)
relu = Dense(5, name='dense1')(bn1)
relu = LeakyReLU(0.1)(relu)
drop1 = Dropout(0.2, name='dropout1')(relu)

bn = BatchNormalization(name='bn1')(drop1)
relu = Dense(5, activation='relu', name='dense2')(bn)
relu = LeakyReLU(0.1)(relu)
drop2 = Dropout(0.2)(relu)

interm = keras.layers.Concatenate()([drop1, drop2])

bn = BatchNormalization(name='bn2')(interm)
relu = Dense(5, activation='relu', name='dense3')(bn)
relu = LeakyReLU(0.1)(relu)
drop = Dropout(0.2)(relu)

interm = keras.layers.Concatenate()([drop, drop2])

bn = BatchNormalization()(interm)
out = Dense(3, activation='softmax', name='dense4')(bn)

model = Model(inputs=inp, outputs=out)

## Standard LR

In [24]:
model.compile(SGD(), loss='categorical_crossentropy', metrics=['accuracy'])

In [25]:
model.fit(x_train, y, validation_data=(np.array(test_df), y_test), epochs=10)

Train on 3501 samples, validate on 514 samples
Epoch 1/10
3501/3501 [==============================] - 4s 1ms/step - loss: 1.0624 - acc: 0.4670 - val_loss: 0.4723 - val_acc: 0.9708
Epoch 2/10
3501/3501 [==============================] - 1s 246us/step - loss: 0.7484 - acc: 0.6455 - val_loss: 0.3128 - val_acc: 0.9669
Epoch 3/10
3501/3501 [==============================] - 1s 218us/step - loss: 0.6174 - acc: 0.7278 - val_loss: 0.2164 - val_acc: 0.9728
Epoch 4/10
3501/3501 [==============================] - 1s 238us/step - loss: 0.5520 - acc: 0.7504 - val_loss: 0.1753 - val_acc: 0.9689
Epoch 5/10
3501/3501 [==============================] - 1s 230us/step - loss: 0.5045 - acc: 0.7772 - val_loss: 0.1567 - val_acc: 0.9689
Epoch 6/10
3501/3501 [==============================] - 1s 215us/step - loss: 0.4511 - acc: 0.8072 - val_loss: 0.1443 - val_acc: 0.9767
Epoch 7/10
3501/3501 [==============================] - 1s 219us/step - loss: 0.4208 - acc: 0.8252 - val_loss: 0.1219 - val_acc: 0.9747
Epo

## LipschitzLR

In [27]:
model.compile(SGD(), loss='categorical_crossentropy', metrics=['accuracy'])

In [28]:
model.fit(x_train, y, validation_data=(np.array(test_df), y_test), epochs=10, callbacks=[lr_scheduler])

Train on 3501 samples, validate on 514 samples
Epoch 1/10
Epoch 0 LR = 0.8341616338518589
3501/3501 [==============================] - 5s 1ms/step - loss: 0.3021 - acc: 0.8732 - val_loss: 0.1181 - val_acc: 0.9864
Epoch 2/10
Epoch 1 LR = 0.8341616338518589
3501/3501 [==============================] - 1s 253us/step - loss: 0.1547 - acc: 0.9400 - val_loss: 0.0658 - val_acc: 0.9903
Epoch 3/10
Epoch 2 LR = 0.8341616338518589
3501/3501 [==============================] - 1s 283us/step - loss: 0.1210 - acc: 0.9532 - val_loss: 0.0982 - val_acc: 0.9883
Epoch 4/10
Epoch 3 LR = 0.8341616338518589
3501/3501 [==============================] - 1s 263us/step - loss: 0.1134 - acc: 0.9660 - val_loss: 0.0886 - val_acc: 0.9883
Epoch 5/10
Epoch 4 LR = 0.8341616338518589
3501/3501 [==============================] - 1s 272us/step - loss: 0.1151 - acc: 0.9572 - val_loss: 0.0518 - val_acc: 0.9903
Epoch 6/10
Epoch 5 LR = 0.8341616338518589
3501/3501 [==============================] - 1s 294us/step - loss: 0.078

# PReLU

In [29]:
inp = Input(shape=(41,))

bn1 = BatchNormalization(name='first_bn')(inp)
relu = Dense(5, name='dense1')(bn1)
relu = PReLU()(relu)
drop1 = Dropout(0.2, name='dropout1')(relu)

bn = BatchNormalization(name='bn1')(drop1)
relu = Dense(5, activation='relu', name='dense2')(bn)
relu = PReLU()(relu)
drop2 = Dropout(0.2)(relu)

interm = keras.layers.Concatenate()([drop1, drop2])

bn = BatchNormalization(name='bn2')(interm)
relu = Dense(5, activation='relu', name='dense3')(bn)
relu = PReLU()(relu)
drop = Dropout(0.2)(relu)

interm = keras.layers.Concatenate()([drop, drop2])

bn = BatchNormalization()(interm)
out = Dense(3, activation='softmax', name='dense4')(bn)

model = Model(inputs=inp, outputs=out)

## Standard LR

In [30]:
model.compile(SGD(), loss='categorical_crossentropy', metrics=['accuracy'])

In [31]:
model.fit(x_train, y, validation_data=(np.array(test_df), y_test), epochs=10)

Train on 3501 samples, validate on 514 samples
Epoch 1/10
3501/3501 [==============================] - 5s 1ms/step - loss: 0.9041 - acc: 0.5990 - val_loss: 0.4179 - val_acc: 0.9183
Epoch 2/10
3501/3501 [==============================] - 1s 219us/step - loss: 0.6454 - acc: 0.7404 - val_loss: 0.3281 - val_acc: 0.9339
Epoch 3/10
3501/3501 [==============================] - 1s 221us/step - loss: 0.5647 - acc: 0.7689 - val_loss: 0.2953 - val_acc: 0.9475
Epoch 4/10
3501/3501 [==============================] - 1s 241us/step - loss: 0.4956 - acc: 0.7963 - val_loss: 0.2709 - val_acc: 0.9591
Epoch 5/10
3501/3501 [==============================] - 1s 233us/step - loss: 0.4422 - acc: 0.8229 - val_loss: 0.2487 - val_acc: 0.9650
Epoch 6/10
3501/3501 [==============================] - 1s 220us/step - loss: 0.3946 - acc: 0.8486 - val_loss: 0.2484 - val_acc: 0.9650
Epoch 7/10
3501/3501 [==============================] - 1s 247us/step - loss: 0.3541 - acc: 0.8638 - val_loss: 0.2393 - val_acc: 0.9669
Epo

## LipschitzLR

In [32]:
model.compile(SGD(), loss='categorical_crossentropy', metrics=['accuracy'])

In [33]:
model.fit(x_train, y, validation_data=(np.array(test_df), y_test), epochs=10, callbacks=[lr_scheduler])

Train on 3501 samples, validate on 514 samples
Epoch 1/10
Epoch 0 LR = 0.8341616338518589
3501/3501 [==============================] - 6s 2ms/step - loss: 0.2391 - acc: 0.9026 - val_loss: 0.2411 - val_acc: 0.9047
Epoch 2/10
Epoch 1 LR = 0.8341616338518589
3501/3501 [==============================] - 1s 251us/step - loss: 0.1439 - acc: 0.9452 - val_loss: 0.0636 - val_acc: 0.9922
Epoch 3/10
Epoch 2 LR = 0.8341616338518589
3501/3501 [==============================] - 1s 284us/step - loss: 0.1088 - acc: 0.9583 - val_loss: 0.0709 - val_acc: 0.9883
Epoch 4/10
Epoch 3 LR = 0.8341616338518589
3501/3501 [==============================] - 1s 271us/step - loss: 0.0956 - acc: 0.9646 - val_loss: 0.0727 - val_acc: 0.9883
Epoch 5/10
Epoch 4 LR = 0.8341616338518589
3501/3501 [==============================] - 1s 292us/step - loss: 0.1035 - acc: 0.9629 - val_loss: 0.0388 - val_acc: 0.9922
Epoch 6/10
Epoch 5 LR = 0.8341616338518589
3501/3501 [==============================] - 1s 295us/step - loss: 0.085

# Dataset with no surface temp related features

In [35]:
df = pd.read_csv('dataset_no_ST_related_features.csv')

In [37]:
train_df, test_df = train_test_split(df, train_size=0.7)
train_df = rebalance(train_df)

y = train_df['hab_lbl']
y_test = test_df['hab_lbl']

train_df.drop('hab_lbl', axis=1, inplace=True)
test_df.drop('hab_lbl', axis=1, inplace=True)

y = to_categorical(np.array(y))
y_test = to_categorical(np.array(y_test))

/home/rahul/.local/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/home/rahul/.local/lib/python3.5/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


# ReLU

In [42]:
inp = Input(shape=(9,))

bn1 = BatchNormalization(name='first_bn')(inp)
relu = Dense(5, activation='relu', name='dense1')(bn1)
drop1 = Dropout(0.2, name='dropout1')(relu)

bn = BatchNormalization(name='bn1')(drop1)
relu = Dense(5, activation='relu', name='dense2')(bn)
drop2 = Dropout(0.2)(relu)

interm = keras.layers.Concatenate()([drop1, drop2])

bn = BatchNormalization(name='bn2')(interm)
relu = Dense(5, activation='relu', name='dense3')(bn)
drop = Dropout(0.2)(relu)

interm = keras.layers.Concatenate()([drop, drop2])

bn = BatchNormalization()(interm)
out = Dense(3, activation='softmax', name='dense4')(bn)

model = Model(inputs=inp, outputs=out)

## Standard LR

In [43]:
lrs = []
model.compile(SGD(), loss='categorical_crossentropy', metrics=['accuracy'])

In [44]:
x_train = np.array(train_df)
batch_size = 32

model.fit(x_train, y, validation_data=(np.array(test_df), y_test), epochs=10)

Train on 7926 samples, validate on 1149 samples
Epoch 1/10
7926/7926 [==============================] - 6s 780us/step - loss: 0.8853 - acc: 0.5193 - val_loss: 0.6336 - val_acc: 0.7946
Epoch 2/10
7926/7926 [==============================] - 2s 208us/step - loss: 0.7402 - acc: 0.6088 - val_loss: 0.5351 - val_acc: 0.8338
Epoch 3/10
7926/7926 [==============================] - 2s 219us/step - loss: 0.6983 - acc: 0.6292 - val_loss: 0.3932 - val_acc: 0.8947
Epoch 4/10
7926/7926 [==============================] - 2s 205us/step - loss: 0.6563 - acc: 0.6411 - val_loss: 0.3311 - val_acc: 0.9130
Epoch 5/10
7926/7926 [==============================] - 2s 210us/step - loss: 0.6406 - acc: 0.6466 - val_loss: 0.2807 - val_acc: 0.9269
Epoch 6/10
7926/7926 [==============================] - 2s 215us/step - loss: 0.6289 - acc: 0.6511 - val_loss: 0.2593 - val_acc: 0.9295
Epoch 7/10
7926/7926 [==============================] - 2s 214us/step - loss: 0.6177 - acc: 0.6518 - val_loss: 0.2492 - val_acc: 0.9304


## LipschitzLR

In [45]:
model.compile(SGD(), loss='categorical_crossentropy', metrics=['accuracy'])

In [47]:
func = K.function([model.layers[0].input], [model.layers[-2].output])

In [48]:
model.fit(x_train, y, validation_data=(np.array(test_df), y_test), epochs=10, callbacks=[lr_scheduler])

Train on 7926 samples, validate on 1149 samples
Epoch 1/10
Epoch 0 LR = 1.2442820769520127
7926/7926 [==============================] - 7s 836us/step - loss: 0.6275 - acc: 0.6474 - val_loss: 0.2005 - val_acc: 0.9347
Epoch 2/10
Epoch 1 LR = 0.6892385140170064
7926/7926 [==============================] - 2s 279us/step - loss: 0.5721 - acc: 0.6747 - val_loss: 0.1350 - val_acc: 0.9556
Epoch 3/10
Epoch 2 LR = 0.9017260115411417
7926/7926 [==============================] - 2s 292us/step - loss: 0.5946 - acc: 0.6648 - val_loss: 0.4779 - val_acc: 0.9077
Epoch 4/10
Epoch 3 LR = 0.7794945472479028
7926/7926 [==============================] - 2s 265us/step - loss: 0.5799 - acc: 0.6842 - val_loss: 0.2903 - val_acc: 0.9225
Epoch 5/10
Epoch 4 LR = 0.7414202820021162
7926/7926 [==============================] - 2s 275us/step - loss: 0.5872 - acc: 0.6877 - val_loss: 0.3124 - val_acc: 0.9199
Epoch 6/10
Epoch 5 LR = 1.0307085808584373
7926/7926 [==============================] - 2s 264us/step - loss: 0.

# LeakyReLU

In [49]:
inp = Input(shape=(9,))

bn1 = BatchNormalization(name='first_bn')(inp)
relu = Dense(5, name='dense1')(bn1)
relu = LeakyReLU(0.1)(relu)
drop1 = Dropout(0.2, name='dropout1')(relu)

bn = BatchNormalization(name='bn1')(drop1)
relu = Dense(5, activation='relu', name='dense2')(bn)
relu = LeakyReLU(0.1)(relu)
drop2 = Dropout(0.2)(relu)

interm = keras.layers.Concatenate()([drop1, drop2])

bn = BatchNormalization(name='bn2')(interm)
relu = Dense(5, activation='relu', name='dense3')(bn)
relu = LeakyReLU(0.1)(relu)
drop = Dropout(0.2)(relu)

interm = keras.layers.Concatenate()([drop, drop2])

bn = BatchNormalization()(interm)
out = Dense(3, activation='softmax', name='dense4')(bn)

model = Model(inputs=inp, outputs=out)

## Standard LR

In [50]:
model.compile(SGD(), loss='categorical_crossentropy', metrics=['accuracy'])

In [51]:
model.fit(x_train, y, validation_data=(np.array(test_df), y_test), epochs=10)

Train on 7926 samples, validate on 1149 samples
Epoch 1/10
7926/7926 [==============================] - 7s 914us/step - loss: 0.9227 - acc: 0.5453 - val_loss: 0.5229 - val_acc: 0.9504
Epoch 2/10
7926/7926 [==============================] - 2s 228us/step - loss: 0.7580 - acc: 0.6427 - val_loss: 0.3963 - val_acc: 0.9330
Epoch 3/10
7926/7926 [==============================] - 2s 225us/step - loss: 0.6939 - acc: 0.6606 - val_loss: 0.3527 - val_acc: 0.9269
Epoch 4/10
7926/7926 [==============================] - 2s 234us/step - loss: 0.6619 - acc: 0.6764 - val_loss: 0.2903 - val_acc: 0.9295
Epoch 5/10
7926/7926 [==============================] - 2s 225us/step - loss: 0.6253 - acc: 0.6944 - val_loss: 0.2624 - val_acc: 0.9278
Epoch 6/10
7926/7926 [==============================] - 2s 234us/step - loss: 0.6252 - acc: 0.6938 - val_loss: 0.2465 - val_acc: 0.9260
Epoch 7/10
7926/7926 [==============================] - 2s 234us/step - loss: 0.6003 - acc: 0.7082 - val_loss: 0.2358 - val_acc: 0.9286


## LipschitzLR

In [52]:
model.compile(SGD(), loss='categorical_crossentropy', metrics=['accuracy'])

In [54]:
func = K.function([model.layers[0].input], [model.layers[-2].output])

In [55]:
model.fit(x_train, y, validation_data=(np.array(test_df), y_test), epochs=10, callbacks=[lr_scheduler])

Train on 7926 samples, validate on 1149 samples
Epoch 1/10
Epoch 0 LR = 0.9730454548336872
7926/7926 [==============================] - 4s 479us/step - loss: 0.5094 - acc: 0.7328 - val_loss: 0.1981 - val_acc: 0.9426
Epoch 2/10
Epoch 1 LR = 0.828665507286387
7926/7926 [==============================] - 2s 282us/step - loss: 0.4994 - acc: 0.7417 - val_loss: 0.2465 - val_acc: 0.9408
Epoch 3/10
Epoch 2 LR = 0.9383622983780305
7926/7926 [==============================] - 2s 288us/step - loss: 0.5318 - acc: 0.7239 - val_loss: 0.0866 - val_acc: 0.9748
Epoch 4/10
Epoch 3 LR = 0.7186669197981984
7926/7926 [==============================] - 2s 284us/step - loss: 0.5221 - acc: 0.7228 - val_loss: 0.3663 - val_acc: 0.9156
Epoch 5/10
Epoch 4 LR = 0.7572325907050088
7926/7926 [==============================] - 2s 281us/step - loss: 0.4950 - acc: 0.7440 - val_loss: 0.1911 - val_acc: 0.9452
Epoch 6/10
Epoch 5 LR = 0.9331614973227133
7926/7926 [==============================] - 2s 283us/step - loss: 0.5

# PReLU

In [56]:
inp = Input(shape=(9,))

bn1 = BatchNormalization(name='first_bn')(inp)
relu = Dense(5, name='dense1')(bn1)
relu = PReLU()(relu)
drop1 = Dropout(0.2, name='dropout1')(relu)

bn = BatchNormalization(name='bn1')(drop1)
relu = Dense(5, activation='relu', name='dense2')(bn)
relu = PReLU()(relu)
drop2 = Dropout(0.2)(relu)

interm = keras.layers.Concatenate()([drop1, drop2])

bn = BatchNormalization(name='bn2')(interm)
relu = Dense(5, activation='relu', name='dense3')(bn)
relu = PReLU()(relu)
drop = Dropout(0.2)(relu)

interm = keras.layers.Concatenate()([drop, drop2])

bn = BatchNormalization()(interm)
out = Dense(3, activation='softmax', name='dense4')(bn)

model = Model(inputs=inp, outputs=out)

## Standard LR

In [57]:
model.compile(SGD(), loss='categorical_crossentropy', metrics=['accuracy'])

In [58]:
model.fit(x_train, y, validation_data=(np.array(test_df), y_test), epochs=10)

Train on 7926 samples, validate on 1149 samples
Epoch 1/10
7926/7926 [==============================] - 8s 1ms/step - loss: 0.9009 - acc: 0.5351 - val_loss: 0.3560 - val_acc: 0.9312
Epoch 2/10
7926/7926 [==============================] - 2s 238us/step - loss: 0.7026 - acc: 0.6342 - val_loss: 0.2939 - val_acc: 0.9382
Epoch 3/10
7926/7926 [==============================] - 2s 230us/step - loss: 0.6528 - acc: 0.6583 - val_loss: 0.2633 - val_acc: 0.9408
Epoch 4/10
7926/7926 [==============================] - 2s 226us/step - loss: 0.6228 - acc: 0.6794 - val_loss: 0.2589 - val_acc: 0.9399
Epoch 5/10
7926/7926 [==============================] - 2s 235us/step - loss: 0.6069 - acc: 0.6827 - val_loss: 0.2458 - val_acc: 0.9399
Epoch 6/10
7926/7926 [==============================] - 2s 245us/step - loss: 0.5960 - acc: 0.6860 - val_loss: 0.2385 - val_acc: 0.9460
Epoch 7/10
7926/7926 [==============================] - 2s 251us/step - loss: 0.5928 - acc: 0.6834 - val_loss: 0.2352 - val_acc: 0.9460
Ep

## LipschitzLR

In [59]:
model.compile(SGD(), loss='categorical_crossentropy', metrics=['accuracy'])

In [60]:
func = K.function([model.layers[0].input], [model.layers[-2].output])

In [61]:
model.fit(x_train, y, validation_data=(np.array(test_df), y_test), epochs=10, callbacks=[lr_scheduler])

Train on 7926 samples, validate on 1149 samples
Epoch 1/10
Epoch 0 LR = 0.8288305687340369
7926/7926 [==============================] - 11s 1ms/step - loss: 0.5947 - acc: 0.6640 - val_loss: 0.1465 - val_acc: 0.9530
Epoch 2/10
Epoch 1 LR = 0.8016015069168954
7926/7926 [==============================] - 2s 314us/step - loss: 0.5623 - acc: 0.6869 - val_loss: 0.1749 - val_acc: 0.9513
Epoch 3/10
Epoch 2 LR = 0.9804867512194028
7926/7926 [==============================] - 3s 324us/step - loss: 0.5673 - acc: 0.6841 - val_loss: 0.1452 - val_acc: 0.9478
Epoch 4/10
Epoch 3 LR = 0.9209644086449322
7926/7926 [==============================] - 3s 340us/step - loss: 0.5524 - acc: 0.6870 - val_loss: 0.1783 - val_acc: 0.9504
Epoch 5/10
Epoch 4 LR = 0.9022393230382826
7926/7926 [==============================] - 3s 333us/step - loss: 0.5388 - acc: 0.7082 - val_loss: 0.1374 - val_acc: 0.9556
Epoch 6/10
Epoch 5 LR = 0.792674659740621
7926/7926 [==============================] - 2s 304us/step - loss: 0.52